In [9]:
import sys
import pandas as pd
import numpy as np
import os
import datetime as DT

sys.path.append("..")
import rbf_functions

In [10]:
rbfs = [
    rbf_functions.original_rbf,
    rbf_functions.squared_exponential_rbf,
    rbf_functions.inverse_quadratic_rbf,
    rbf_functions.inverse_multiquadric_rbf,
    rbf_functions.exponential_rbf,
    rbf_functions.matern32_rbf,
    rbf_functions.matern52_rbf,
]

# Load model output for each activation function

In [11]:
# output_dir = f"../output/{rbf}/"
nfearchive = {}
for n in rbfs:
    nfearchive[n.__name__] = {}
for entry in rbfs:
    name = entry.__name__
    output_dir = f"../output/{name}/"
    for i in os.listdir(output_dir):
        if i.endswith("_hypervolume.csv"):
            #             nfearchive[name][i.split("_")[0]] = pd.read_csv(output_dir + i)
            archives_by_nfe = pd.read_csv(output_dir + i)
            nfes = archives_by_nfe["Unnamed: 0"].values
            u_nfes = np.unique(nfes)
            #             selected_nfe = u_nfes[-1::]
            selected_nfe = u_nfes[0::10]
            selected_nfe = np.append(selected_nfe, u_nfes[-1::])
            nfearchive[name][int(i.split("_")[0])] = archives_by_nfe.loc[
                archives_by_nfe["Unnamed: 0"].isin(selected_nfe)
            ]

# Transform the dataframe back into platypus solution instances, to have a collection of solutions for each generation

In [12]:
from platypus import Solution, Problem, Hypervolume

n_rbfs = 4
n_objs = 6
n_vars = n_rbfs * 8

problem = Problem(n_vars, n_objs)

# matters for hypervolume
problem.directions[0] = Problem.MAXIMIZE  # hydropower
problem.directions[1] = Problem.MAXIMIZE  # atomic power plant
problem.directions[2] = Problem.MAXIMIZE  # baltimore
problem.directions[3] = Problem.MAXIMIZE  # chester
problem.directions[4] = Problem.MINIMIZE  # environment
problem.directions[5] = Problem.MAXIMIZE  # recreation

archives = {}
seeds = np.arange(10, 101, 10).tolist()
for n in rbfs:
    archives[n.__name__] = {}
    for i in seeds:
        archives[n.__name__][i] = {}
for entry in rbfs:
    name = entry.__name__
    for s in nfearchive[name]:  # nfearchive[name]:
        for nfe, generation in nfearchive[name][s].groupby("Unnamed: 0"):
            # we slice from 2, getting rid of the first two columns
            # which contain the NFE and ID of each solution
            archive = []
            for i, row in generation.iloc[:, 2::].iterrows():
                solution = Solution(problem)
                solution.objectives = row
                archive.append(solution)
            archives[name][s][nfe] = archive

# Load reference sets, turn into solutions sets

In [14]:
ref_dir = "./refsets/"
ref_sets = {}
for n in rbfs:
    name = n.__name__
    ref_sets[name] = {}
    data = pd.read_csv(f"{ref_dir}{name}_refset.csv")
    ref_set = []
    for i, row in data.iterrows():
        solution = Solution(problem)
        solution.objectives = row
        ref_set.append(solution)
    ref_sets[name] = ref_set

In [15]:
ref_sets.keys()

dict_keys(['original_rbf', 'squared_exponential_rbf', 'inverse_quadratic_rbf', 'inverse_multiquadric_rbf', 'exponential_rbf', 'matern32_rbf', 'matern52_rbf'])

# Or, load global reference set into solution set

In [16]:
ref_dir = "./refsets"
data = pd.read_csv(f"{ref_dir}/global_refset.csv")
ref_set = []
for i, row in data.iterrows():
    solution = Solution(problem)
    solution.objectives = row
    ref_set.append(solution)

# Calculate Hypervolume for each seed

In [ ]:
rbf = "original_rbf"
tempnfe = {}
temphv = {}
nfe_sets = {}
hv_sets = {}
# for rbf in archives:
nfe_sets[rbf] = {}
hv_sets[rbf] = {}
hv = Hypervolume(reference_set=ref_sets[rbf])
# hv = Hypervolume(reference_set=ref_set) #global
print(f"started {rbf} at {DT.datetime.now().strftime('%H:%M:%S')}")
# seedl = [10, 20, 30]
for seed in archives[rbf]:
    nfe_sets[rbf][seed] = {}
    hv_sets[rbf][seed] = {}
    s_archives = archives[rbf][seed]
    nfes = []
    hvs = []
    for nfe, archive in s_archives.items():
        nfes.append(nfe)
        hvs.append(hv.calculate(archive))
    nfe_sets[rbf][seed] = nfes
    hv_sets[rbf][seed] = hvs
    tempnfe[seed] = nfes
    temphv[seed] = hvs
    dfhv = pd.DataFrame.from_dict(temphv, orient="index")
    dfnfe = pd.DataFrame.from_dict(tempnfe, orient="index")
    dfhv = dfhv.T
    dfnfe = dfnfe.T
    dfhv.to_csv(f"hv/{rbf}_hv.csv", index=False)
    dfnfe.to_csv(f"hv/{rbf}_nfe.csv", index=False)
    #     dfhv.to_csv(f"hv_global/{rbf}_hv_all.csv", index=False) #global
    #     dfnfe.to_csv(f"hv_global/{rbf}_nfe_all.csv", index=False) #global
    print(f"finished seed: {seed} at {DT.datetime.now().strftime('%H:%M:%S')}")

started original_rbf at 13:05:01


In [ ]:
rbf = "original_rbf"

fig, ax = plt.subplots(sharex=True)
for i in nfe_sets_all:
    ax.plot(nfe_sets_all[i], hv_sets_all[i])

ax.set_xlabel("nfe")
ax.set_ylabel("hypervolume")
ax.set_title(rbf)
sns.despine()

plt.show()

# Plot Hypervolume (local reference set) 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={"figure.figsize": (11.7, 8.27)})
sns.set_style("white")

# Plot hypervolume for each rbf

for entry, c in zip(rbfs, sns.color_palette()):
    rbf = entry.__name__
    df_hv = pd.read_csv(f"hv/{rbf}_hv.csv")
    df_hv_nfe = pd.read_csv(f"hv/{rbf}_hv_nfe.csv")
    fig, ax = plt.subplots(sharex=True)

    ax.plot(df_hv_nfe, df_hv, color=c)

    ax.set_xlabel("nfe")
    ax.set_ylabel("hypervolume")
    ax.set_title(rbf)
    sns.despine()
#     plt.savefig(f'figs/{rbf}/{rbf}_hv.jpg')
plt.show()

# Plot Epsilon progress

In [ ]:
for entry, c in zip(rbfs, sns.color_palette()):
    rbf = entry.__name__

    ep_nfe = {}
    ep_improv = {}
    output_dir = f"../output/{rbf}/"
    for i in os.listdir(output_dir):
        if i.endswith("_convergence.csv"):
            ep_load = pd.read_csv(output_dir + i)
            ep_nfe[int(i.split("_")[0])] = ep_load.nfe
            ep_improv[int(i.split("_")[0])] = ep_load.improvements
    ep_nfe = pd.DataFrame.from_dict(ep_nfe)
    ep_improv = pd.DataFrame.from_dict(ep_improv)

    fig, ax = plt.subplots(sharex=True)
    ax.plot(ep_nfe, ep_improv, color=c)

    ax.set_xlabel("nfe")
    ax.set_ylabel("Epsilon progress")
    ax.set_title(rbf)
    sns.despine()
#     plt.savefig(f'figs/{rbf}/{rbf}_epsprog.jpg')
plt.show()

# Plot Archive Size

In [ ]:
for entry, c in zip(rbfs, sns.color_palette()):
    rbf = entry.__name__
    archive_nfe = {}
    archive_count = {}
    output_dir = f"../output/{rbf}/"
    for i in os.listdir(output_dir):
        if i.endswith("_hypervolume.csv"):
            archives_by_nfe = pd.read_csv(output_dir + i)
            dups = archives_by_nfe.pivot_table(columns=["Unnamed: 0"], aggfunc="size")
            archive_nfe[int(i.split("_")[0])] = dups.index
            archive_count[int(i.split("_")[0])] = dups.reset_index(drop=True)

    archive_nfe = pd.DataFrame.from_dict(archive_nfe, orient="index").T
    archive_size = pd.DataFrame.from_dict(archive_count, orient="index").T

    fig, ax = plt.subplots(sharex=True)
    ax.plot(archive_nfe, archive_size, color=c)

    ax.set_xlabel("nfe")
    ax.set_ylabel("Archive size")
    ax.set_title(rbf)
    sns.despine()
#     plt.savefig(f'figs/{rbf}/{rbf}_archsize.jpg')
plt.show()

# Plot Hypervolume (global reference set) 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={"figure.figsize": (11.7, 8.27)})
sns.set_style("white")

# Plot hypervolume for each rbf

for entry, c in zip(rbfs, sns.color_palette()):
    rbf = entry.__name__
    df_hv = pd.read_csv(f"hv_global/{rbf}_hv_all.csv")
    df_hv_nfe = pd.read_csv(f"hv_global/{rbf}_hv_nfe_all.csv")
    fig, ax = plt.subplots(sharex=True)

    ax.plot(df_hv_nfe, df_hv, color=c)

    ax.set_xlabel("nfe")
    ax.set_ylabel("hypervolume")
    ax.set_title(rbf)
    sns.despine()
#     plt.savefig(f'figs/{rbf}/{rbf}_hv_global.jpg')
plt.show()